In [1]:
!pip install "tensorflow_hub"

In [1]:
!pip install "tensorflow==2.11.*"
!pip uninstall -y tensorflow-io
!pip install "tensorflow_io==0.28.*"

     ---------------------------------------- 0.0/266.3 MB ? eta -:--:--
     --------------------------------------- 1.0/266.3 MB 33.4 MB/s eta 0:00:08
     --------------------------------------- 1.0/266.3 MB 33.4 MB/s eta 0:00:08
     --------------------------------------- 1.0/266.3 MB 33.4 MB/s eta 0:00:08
     ---------------------------------------- 1.4/266.3 MB 7.5 MB/s eta 0:00:36
     --------------------------------------- 3.1/266.3 MB 14.4 MB/s eta 0:00:19
     --------------------------------------- 3.1/266.3 MB 14.4 MB/s eta 0:00:19
     --------------------------------------- 3.1/266.3 MB 14.4 MB/s eta 0:00:19
     --------------------------------------- 3.1/266.3 MB 14.4 MB/s eta 0:00:19
      -------------------------------------- 5.2/266.3 MB 12.9 MB/s eta 0:00:21
      -------------------------------------- 5.2/266.3 MB 12.9 MB/s eta 0:00:21
      -------------------------------------- 5.2/266.3 MB 12.9 MB/s eta 0:00:21
      -------------------------------------- 5.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-datasets 4.9.1 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.


     ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
     ---------------------------------------- 0.2/22.9 MB 4.6 MB/s eta 0:00:05
      --------------------------------------- 0.5/22.9 MB 5.2 MB/s eta 0:00:05
     - -------------------------------------- 1.0/22.9 MB 6.7 MB/s eta 0:00:04
     - -------------------------------------- 1.0/22.9 MB 7.3 MB/s eta 0:00:03
     - -------------------------------------- 1.0/22.9 MB 7.3 MB/s eta 0:00:03
     -- ------------------------------------- 1.5/22.9 MB 5.6 MB/s eta 0:00:04
     --- ------------------------------------ 2.1/22.9 MB 6.6 MB/s eta 0:00:04
     --- ------------------------------------ 2.1/22.9 MB 6.6 MB/s eta 0:00:04
     --- ------------------------------------ 2.1/22.9 MB 5.2 MB/s eta 0:00:04
     ----- ---------------------------------- 2.9/22.9 MB 6.3 MB/s eta 0:00:04
     ------ --------------------------------- 3.4/22.9 MB 6.9 MB/s eta 0:00:03
     ------ --------------------------------- 3.9/22.9 MB 7

In [6]:
from google.colab import files
!rm -f kaggle*.json
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d soumendraprasad/musical-instruments-sound-dataset

100% 5.40G/5.40G [00:49<00:00, 144MB/s]
100% 5.40G/5.40G [00:49<00:00, 117MB/s]


In [ ]:
!mkdir -p ./datasets/musical-instruments-sound-dataset
!unzip musical-instruments-sound-dataset.zip -d ./datasets/musical-instruments-sound-dataset

# convertendo para 16 bit


In [ ]:
import soundfile as sf
import os


def convert16(source_path, destination_path):
  #create destination folder if not exists
  if not os.path.exists(destination_path): 
    os.makedirs(destination_path)

  #Listar todos os arquivos no diretório
  file_list = os.listdir(source_path)

  for item in file_list: 
    source_fullpath = os.path.join(source_path, item); 
    dest_fullpath = os.path.join(destination_path, item); 
    data, samplerate = sf.read(source_fullpath); 
    sf.write(dest_fullpath, data, samplerate, subtype='PCM_16');




In [ ]:
#convertendo pasta train para 16bit
source_path = './datasets/musical-instruments-sound-dataset/Train_submission/Train_submission/'; 
destination_path = './datasets/musical-instruments-sound-dataset/Train16';

convert16(source_path, destination_path)

In [ ]:
#convertendo pasta test para 16bit
source_path = './datasets/musical-instruments-sound-dataset/Test_submission/Test_submission/'; 
destination_path = './datasets/musical-instruments-sound-dataset/Test16';

convert16(source_path, destination_path)

In [5]:
#fazendo download dos arquivos para uma pasta local que sera usada durante o treinamento
!mkdir -p /content/datasets
!cp -R "/content/drive/MyDrive/senac/RNAP/grupo projeto/dataset-16bit" /content/datasets

A sintaxe do comando est� incorreta.
'-R' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [6]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8017396410696879292
xla_global_id: -1
]


In [2]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [3]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [4]:
train_csv = './dataset-16bit/Metadata_Train.csv'
base_data_path = './dataset-16bit/Train16'

pd_data = pd.read_csv(train_csv)
pd_data['FileName'] = pd_data['FileName'].apply(lambda row: os.path.join(base_data_path, row))
pd_data.head(10)

,FileName,Class
0,./dataset-16bit/Train16\1-E1-Major 00.wav,Sound_Guitar
1,./dataset-16bit/Train16\1-E1-Major 01.wav,Sound_Guitar
2,./dataset-16bit/Train16\1-E1-Major 02.wav,Sound_Guitar
3,./dataset-16bit/Train16\1-E1-Major 03.wav,Sound_Guitar
4,./dataset-16bit/Train16\1-E1-Major 04.wav,Sound_Guitar
5,./dataset-16bit/Train16\1-E1-Major 05.wav,Sound_Guitar
6,./dataset-16bit/Train16\1-E1-Major 06.wav,Sound_Guitar
7,./dataset-16bit/Train16\1-E1-Major 07.wav,Sound_Guitar
8,./dataset-16bit/Train16\1-E1-Major 08.wav,Sound_Guitar
9,./dataset-16bit/Train16\1-E1-Major 09.wav,Sound_Guitar


In [5]:
my_classes = pd_data['Class'].unique()
my_classes

array(['Sound_Guitar', 'Sound_Drum', 'Sound_Violin', 'Sound_Piano'],
      dtype=object)

In [6]:
map_class_to_id = {class_name: index for index, class_name in enumerate(my_classes)}
map_class_to_id

{'Sound_Guitar': 0, 'Sound_Drum': 1, 'Sound_Violin': 2, 'Sound_Piano': 3}

In [7]:
class_id = pd_data['Class'].apply(lambda name: map_class_to_id[name])
pd_data = pd_data.assign(target=class_id)
pd_data.head(10)

,FileName,Class,target
0,./dataset-16bit/Train16\1-E1-Major 00.wav,Sound_Guitar,0
1,./dataset-16bit/Train16\1-E1-Major 01.wav,Sound_Guitar,0
2,./dataset-16bit/Train16\1-E1-Major 02.wav,Sound_Guitar,0
3,./dataset-16bit/Train16\1-E1-Major 03.wav,Sound_Guitar,0
4,./dataset-16bit/Train16\1-E1-Major 04.wav,Sound_Guitar,0
5,./dataset-16bit/Train16\1-E1-Major 05.wav,Sound_Guitar,0
6,./dataset-16bit/Train16\1-E1-Major 06.wav,Sound_Guitar,0
7,./dataset-16bit/Train16\1-E1-Major 07.wav,Sound_Guitar,0
8,./dataset-16bit/Train16\1-E1-Major 08.wav,Sound_Guitar,0
9,./dataset-16bit/Train16\1-E1-Major 09.wav,Sound_Guitar,0


In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(pd_data, test_size=0.2, random_state=42)

In [9]:
train_data

,FileName,Class,target
1208,./dataset-16bit/Train16\WaveDrum02_43HH (47).wav,Sound_Drum,1
2356,./dataset-16bit/Train16\ROOM_room4_MUS_bartok_...,Sound_Piano,3
482,./dataset-16bit/Train16\latin_4_130BPM.wav,Sound_Guitar,0
1199,./dataset-16bit/Train16\WaveDrum02_43HH (39).wav,Sound_Drum,1
438,./dataset-16bit/Train16\G53-70606-1111-306.wav,Sound_Guitar,0
...,...,...,...
1638,./dataset-16bit/Train16\RealDrum01_06KDtrain.wav,Sound_Violin,2
1095,./dataset-16bit/Train16\WaveDrum02_39KD (65).wav,Sound_Drum,1
1130,./dataset-16bit/Train16\WaveDrum02_39KD (97).wav,Sound_Drum,1
1294,./dataset-16bit/Train16\WaveDrum02_45SD (124).wav,Sound_Drum,1


In [10]:
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [11]:
train_filenames = train_data['FileName']
train_targets = train_data['target']

train_ds = tf.data.Dataset.from_tensor_slices((train_filenames, train_targets))
train_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [12]:
test_filenames = test_data['FileName']
test_targets = test_data['target']

val_ds = tf.data.Dataset.from_tensor_slices((test_filenames, test_targets))
val_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [13]:
def load_wav_for_map(filename, label):
  return load_wav_16k_mono(filename), label

train_ds = train_ds.map(load_wav_for_map)
val_ds = val_ds.map(load_wav_for_map)
train_ds.element_spec

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [14]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings))

# extract embedding
train_ds = train_ds.map(extract_embedding).unbatch()
val_ds = val_ds.map(extract_embedding).unbatch()
train_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [15]:
train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [23]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(350, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 350)               179550    
                                                                 
 dense_9 (Dense)             (None, 300)               105300    
                                                                 
 dense_10 (Dense)            (None, 200)               60200     
                                                                 
 dense_11 (Dense)            (None, 4)                 804       
                                                                 
Total params: 870,654
Trainable params: 870,654
Non-trainable params: 0
_________________________________________________________________


In [24]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [25]:
history = my_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/20
2439/2439 [==============================] - 30s 12ms/step - loss: 0.3913 - accuracy: 0.8107 - val_loss: 0.6572 - val_accuracy: 0.7157
Epoch 2/20
2439/2439 [==============================] - 27s 11ms/step - loss: 0.3559 - accuracy: 0.8247 - val_loss: 0.7079 - val_accuracy: 0.7268
Epoch 3/20
2439/2439 [==============================] - 27s 11ms/step - loss: 0.3320 - accuracy: 0.8340 - val_loss: 0.7502 - val_accuracy: 0.7166
Epoch 4/20
2439/2439 [==============================] - 27s 11ms/step - loss: 0.3148 - accuracy: 0.8464 - val_loss: 0.6585 - val_accuracy: 0.7263
Epoch 5/20
2439/2439 [==============================] - 28s 11ms/step - loss: 0.2994 - accuracy: 0.8509 - val_loss: 0.7624 - val_accuracy: 0.7408
Epoch 6/20
2439/2439 [==============================] - 27s 11ms/step - loss: 0.2887 - accuracy: 0.8582 - val_loss: 0.8119 - val_accuracy: 0.7333
Epoch 7/20
2439/2439 [==============================] - 28s 12ms/step - loss: 0.2808 - accuracy: 0.8604 - val_loss: 0.9532 -

In [26]:
# my_model.save('./music-instruments-sounds') # é possivel apenas realizar o load dos pesos
my_model.save_weights('./music-instruments-sounds-weights2')

In [ ]:
!zip -r ./music-instruments-sounds.zip ./music-instruments-sounds

  adding: music-instruments-sounds/ (stored 0%)
  adding: music-instruments-sounds/saved_model.pb (deflated 87%)
  adding: music-instruments-sounds/fingerprint.pb (stored 0%)
  adding: music-instruments-sounds/assets/ (stored 0%)
  adding: music-instruments-sounds/keras_metadata.pb (deflated 83%)
  adding: music-instruments-sounds/variables/ (stored 0%)
  adding: music-instruments-sounds/variables/variables.index (deflated 57%)
  adding: music-instruments-sounds/variables/variables.data-00000-of-00001 (deflated 17%)


In [ ]:
from google.colab import files
files.download('./music-instruments-sounds.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
testing_wav_file_name = '/content/datasets/musical-instruments-sound-dataset/Test16/pop-drums-loops-3-11278.wav';
testing_wav_data = load_wav_16k_mono(testing_wav_file_name)
display.Audio(testing_wav_data, rate=16000)

In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
result = my_model(embeddings).numpy()

inferred_class = my_classes[result.mean(axis=0).argmax()]
inferred_class

NameError: ignored

In [ ]:
yamlnet

In [ ]:
my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 526,852
Trainable params: 526,852
Non-trainable params: 0
_________________________________________________________________


In [ ]:
yamnet_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fbac2902730>

In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.argmax(class_scores)
print(top_class.value_index)
#inferred_class = my_classes[top_class]

#print(f'The main sound is: {inferred_class}')
#print(f'The embeddings shape: {embeddings.shape}')

AttributeError: ignored